# Postupci nelinearne optimizacije

In [56]:
from Matrica import Matrica, JedinicnaMatrica
from functools import reduce
import math
import numpy as np

In [57]:
# Ulazne velicine:
# - tocka: pocetna tocka pretrazivanja
# - h: pomak pretrazivanja
# - f: ciljna funkcija


def unimodalni(x0, func, h_value = 1.0):
    
    h = Matrica(1, 1, [h_value])
    tocka = x0.copy()
    left = tocka - h 
    right = tocka + h
    m = tocka
    step = 1
    
    fm = func.vrijednost(tocka)
    fl = func.vrijednost(left)
    fr = func.vrijednost(right)
    
    if fm < fr and fm < fl: 
        return left, right
    elif fm > fr:
        while True:
            left = m
            m = right
            fm = fr
            step *= 2
            right = tocka + (h * step)
            fr = func.vrijednost(right)
            if (fm <= fr): 
                break
    else: 
        while True:
            right = m
            m = left
            fm = fl
            step *= 2
            left = tocka - (h * step)
            fl = func.vrijednost(left)
            if fm <= fl: break
    return left, right

In [58]:
# Ulazne velicine:
# - a, b: pocetne granice unimodalnog intervala
# - e: preciznost

def zlatni_rez(a, b, func, epsilon = 1e-6, max_iter = 20000):
    
    k = 0.5 * (math.sqrt(5) - 1)
        
    c = b - k * (b - a)
    d = a + k * (b - a)
    fc = func.vrijednost(c)
    fd = func.vrijednost(d)
    
    br_iter = 0
    while((b[0][0] - a[0][0]) > epsilon and br_iter <= max_iter):
        if fc < fd:
            b = d 
            d = c
            c = b - k * (b - a)
            fd = fc
            fc = func.vrijednost(c)
        else:
            a = c
            c = d
            d = a + k * (b - a)
            fc = fd
            fd = func.vrijednost(d)
        br_iter += 1
    return (a + b) / 2

In [59]:
def koord_pretr(x0, func, epsilon = 1e-6, max_iter = 200):
    n = x0.br_stup
    x = x0.copy()
    
    e = Matrica(1, n, np.zeros(n))
#     eps = Matrica(1, n, np.full((1, n), epsilon))
    
    br_iter = 0
    while True:
        br_iter +=1
        xs = x.copy()
        for i in range(n):
            e[i] = 1
            a, b = unimodalni(xs, func)
            l_min = zlatni_rez(a, b, func, epsilon) 
            x += l_min*e
            e[i] = 0
            for i in range(x.br_stup):
                if norma(x, xs) > epsilon: 
                    break
                return x
        if br_iter > max_iter:
            return x
        
def norma(x1, x2):
    suma = 0
    for i in range(x1.br_stup):
        suma += (x1[0][i] - x2[0][i])**2
    rezultat = suma**(1/2)
    return rezultat
        

# def pretraga(elementi):
#     l = elementi[0]
#     x[i][0] += l
#     rezultat = func.vrijednost(x)
#     x[i][0] -= l
#     return rezultat
    


In [60]:
def simplex(x0, func, pomak = 1, alfa = 1.0, beta = 0.5, gama = 2.0, sigma = 0.5, epsilon=1e-6, iter_limit = 2000):
    
    #tocke simplexa
    pocetna_tocka = x0.copy()
    X = [pocetna_tocka]
    for i in range(x0.br_stup):
        nova_tocka = x0.copy()
        nova_tocka[0][i] += pomak
        X.append(nova_tocka)
    
    # - h, l, centroid -
    iteration = 0
    while True:
        iteration += 1
        h = index_min(X, func)
        l = index_max(X, func)
        xc = centroid(X, h)
        xr = refleksija(xc, X[h], alfa)
        
        if func.vrijednost(xr) < func.vrijednost(X[l]):
            xe = ekspanzija(xr, xc, gama)
            
            if func.vrijednost(xe) < func.vrijednost(X[l]):
                X[h] = xe.copy()
            else:
                X[h] = xr.copy()

        else:
            # provjera
            flag = True
            for j in range(len(X)): 
                if j != h:
                    if func.vrijednost(xr) < func.vrijednost(X[j]):
                        flag = False
                        break
            if flag:
                if func.vrijednost(xr) < func.vrijednost(X[h]):
                    X[h] = xr.copy()

                xk = kontrakcija(X[h], xc, beta)

                if func.vrijednost(xk) < func.vrijednost(X[h]):
                    X[h] = xk.copy()
                else:
                    pomak_prema_xl(X, l, sigma)
            else:
                X[h] = xr.copy()
                
        # Uvjet zaustavljanja
        s = 0
        for i in range(len(X)):
            s += (func.vrijednost(X[i]) - func.vrijednost(xc))**2
#         print(X[l], X[h], xc)
        
        if math.sqrt(s / float(len(X) - 1)) <= epsilon or iteration > iter_limit:
            return xc
            
        
# - MINIMUM I MAKSIMUM FJE CILJA        
def index_min(X, func):
    i_min = 0
    for i in range(len(X)):
        if func.vrijednost(X[i]) < func.vrijednost(X[i_min]): i_min = i
    return i_min
 
def index_max(X, func):
    i_max = 0
    for i in range(len(X)):
        if func.vrijednost(X[i]) > func.vrijednost(X[i_max]): i_max = i
    return i_max

# - CENTROID -
def centroid(X, h):
    tocke = X.copy()
    del tocke[h]
    
    if len(tocke) == 1:
        centroid = tocke[0].elementi
    else: 
        red = reduce(myadd, tocke)
        centroid = (1.0 / len(tocke)) * red
    return Matrica(1, X[0].br_stup, centroid[0])

def myadd(a, b):
    return Matrica(a.br_red, a.br_stup, a.elementi + b.elementi, True)

# - REFLEKSIJA, EKSPANZIJA, KONTRAKCIJA -
def refleksija(xc, xh, alfa = 1.0):
    return ((1.0 + alfa) * xc) - (alfa * xh)

def ekspanzija(xc, xr, gama = 2.0):
    return ((1.0 - gama) * xc) + (gama * xr)

def kontrakcija(xh, xc, beta = 0.5):
    return ((1.0 - beta) * xc) + (beta * xh)

def pomak_prema_xl(X, l, sigma = 0.5):
    for i in range(len(X)):
        X[i] = sigma * (X[i] + X[l])
    return X

In [61]:
# x0 - početna točka
# xb - bazna točka
# xp - početna točka pretraživanja
# xn - točka dobivena pretraživanjem

def hooke_jeeves(x0, func, dx = 0.5, epsilon = 1e-6):

    xp = x0.copy()
    xb = x0.copy()
    dx = Matrica(x0.br_red, x0.br_stup, np.full((x0.br_red, x0.br_stup), 0.5), True)
                 
    while True:
        xn = istrazi(func, xp, dx)
        
#         print(xb[0][0], "\t", xp[0][0], "\t", xn[0][0])
#         print("{:4.2f} \t {:4.2f} \t {:4.2f}".format(func.vrijednost(xb), func.vrijednost(xp), func.vrijednost(xn)))
#         print("------------------------------------------")
        
        if func.vrijednost(xn) < func.vrijednost(xb):
            xp = 2*xn - xb
            xb = xn.copy()
        else:
            dx = dx / 2
            xp = xb.copy()
    
        #uvjet zaustavljanja
        if dx[0][0] < epsilon:
            return xb
        
        
def istrazi(func, xp, dx):
    x = xp.copy()
    for i in range(x.br_stup):
        p = func.vrijednost(x)
        x += dx
        n = func.vrijednost(x)
        if n > p:
            x -= 2 * dx
            n = func.vrijednost(x)
            if n > p:
                x += dx
    return x

In [62]:
class Funkcija:
    
    def __init__(self, f):
        self.br_poziva = 0
        self.f = f
        self.vrijednosti = {}
    
    def brojac(self):
        return self.br_poziva
    
    def reset(self):
        self.br_poziva = 0
    
    def vrijednost(self, x):
#         print(x)
        vrijednost = self.f(x)
#         print(vrijednost)
        if str(x) not in self.vrijednosti:
            self.vrijednosti[str(x)] = vrijednost
        self.br_poziva += 1
        return vrijednost

def f1(vektor):
    x = vektor.elementi.flatten()
    return 100 * (x[1] - (x[0])**2)**2 + (1 - x[0])**2

def f2(vektor):
    x = vektor.elementi.flatten()
    return (x[0] - 4)**2 + 4 * (x[1] - 2)**2

def f3(vektor):
    x = vektor.elementi.flatten()
    result = 0
    for i in range(len(x)):
        result += (x[i] - i + 1)**2
    return result

def f4(vektor):
    x = vektor.elementi.flatten()
    return abs((x[0] - x[1]) * (x[0] + x[1])) + math.sqrt(x[0]**2 + x[1]**2)

def f6(v):
    x = v.elementi.flatten()
    suma = 0
    try:
        for i in range(len(x)):
            suma += x[i]**2
    except TypeError: 
        suma = v**2
    sinus = math.sin(math.sqrt(suma))
    razlomak = ((sinus**2) - 0.5) / ((1 + 0.001 * suma)**2)
    return 0.5 + razlomak

## Zadaci

1. Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sve postupke na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [63]:
#pronadeni minimum i broj evaluacija za svaki pojedini postupak
#sto vise se udaljavati od tocke te ponovno pokretati postupak
for i in range(10, 500, 50):
    x0 = Matrica(1, 1, [[i]], True)

    func_hj = Funkcija(f3)
    func_sim = Funkcija(f3)
    func_koord = Funkcija(f3)
    func_zl = Funkcija(f3)

    a, b = unimodalni(x0, func_sim)
    
    print("pocetna tocka je: " + str(i))
    hj = hooke_jeeves(x0, func_hj)
    print(func_hj.br_poziva, hj, "hj")

    sim = simplex(x0, func_sim)
    print(func_sim.br_poziva, sim, "sim")

    zl = zlatni_rez(a, b, func_zl)
    print(func_zl.br_poziva, zl, "zl")

    koord = koord_pretr(x0, func_koord)
    print(func_koord.br_poziva, koord)
    print("-----------------------------")

pocetna tocka je: 10
133 [[-1.]] hj
507 [[11.]] sim
36 [[-1.00000014]] zl
10067 [[-190.99999874]]
-----------------------------
pocetna tocka je: 60
182 [[-1.]] hj
550 [[61.]] sim
41 [[-1.0000002]] zl
9834 [[-141.00000145]]
-----------------------------
pocetna tocka je: 110
219 [[-1.]] hj
571 [[111.]] sim
42 [[-1.00000018]] zl
9781 [[-90.9999985]]
-----------------------------
pocetna tocka je: 160
227 [[-1.]] hj
591 [[161.]] sim
42 [[-0.9999999]] zl
9905 [[-40.99999924]]
-----------------------------
pocetna tocka je: 210
271 [[-1.]] hj
592 [[211.]] sim
44 [[-0.9999998]] zl
10393 [[8.99999991]]
-----------------------------
pocetna tocka je: 260
270 [[-1.]] hj
592 [[261.]] sim
44 [[-1.00000012]] zl
10788 [[58.99999604]]
-----------------------------
pocetna tocka je: 310
298 [[-1.]] hj
612 [[311.]] sim
44 [[-1.00000004]] zl
11013 [[109.00000099]]
-----------------------------
pocetna tocka je: 360
316 [[-1.]] hj
612 [[361.]] sim
44 [[-0.99999996]] zl
11163 [[159.00000221]]
----------

2. Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1 - 4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5).
Za svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata? 

In [64]:
pocetne_tocke = [[-1.9, 2], [0.1, 0.3], [0, 0, 0, 0, 0], [5.1, 1.1]]
minimum = [[1, 1], [4, 2], [1, 2, 3, 4, 5], [0, 0]]
f = [f1, f2, f3, f4]

for i in range(4):
    
    print("f{0} \t pocetna tocka: {1} \t minimum: {2}".format(i, pocetne_tocke[i], minimum[i]))
    x0 = Matrica(1, len(pocetne_tocke[i]), pocetne_tocke[i])

    func_hj = Funkcija(f[i])
    func_sim = Funkcija(f[i])
    func_zl = Funkcija(f[i])

    a, b = unimodalni(x0, func_sim)
    
    hj = hooke_jeeves(x0, func_hj)
    print(func_hj.br_poziva, "\t", hj, "\t", "hj")

    sim = simplex(x0, func_sim)
    print(func_sim.br_poziva, "\t",sim, "\t", "sim")

    zl = zlatni_rez(a, b, func_zl)
    print(func_zl.br_poziva, "\t",zl, "\t", "zl")
    print("-------------------------------------------------------------------")

f0 	 pocetna tocka: [-1.9, 2] 	 minimum: [1, 1]
236 	 [[-1.53562584  2.36437416]] 	 hj
44025 	 [[-1.4  2. ]] 	 sim
33 	 [[-0.90000033 -0.90000033]] 	 zl
-------------------------------------------------------------------
f1 	 pocetna tocka: [0.1, 0.3] 	 minimum: [4, 2]
222 	 [[2.23999939 2.43999939]] 	 hj
649 	 [[0.10000012 0.3       ]] 	 sim
33 	 [[2.40000007 2.40000007]] 	 zl
-------------------------------------------------------------------
f2 	 pocetna tocka: [0, 0, 0, 0, 0] 	 minimum: [1, 2, 3, 4, 5]
338 	 [[1. 1. 1. 1. 1.]] 	 hj
1200 	 [[9.99999809e-01 4.76837158e-08 4.76837158e-08 4.76837158e-08
  0.00000000e+00]] 	 sim
33 	 [[1.00000021 1.00000021 1.00000021 1.00000021 1.00000021]] 	 zl
-------------------------------------------------------------------
f3 	 pocetna tocka: [5.1, 1.1] 	 minimum: [0, 0]
229 	 [[ 1.99999962 -2.00000038]] 	 hj
676 	 [[6.09999994 1.1       ]] 	 sim
33 	 [[7.09999967 7.099999  ]] 	 zl
-----------------------------------------------------------------

3. Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate! 

In [65]:
x0 = Matrica(1, 2, [5, 5])
# print(x0)
func_hj = Funkcija(f4)
func_sim = Funkcija(f4)

print(hooke_jeeves(x0, func_hj))
print(simplex(x0, func_sim))

[[0. 0.]]
[[5.5 5.5]]


4. Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

In [66]:
x0 = Matrica(1, 2, [0.5, 0.5])
x1 = Matrica(1, 2, [20, 20])

for pomak in range(1, 21):
    
    func_sim1 = Funkcija(f1)
    func_sim2 = Funkcija(f1)

    y1 = simplex(x0, func_sim1, pomak)
    y2 = simplex(x1, func_sim2, pomak)
    
    print("pomak:", pomak, "\t", func_sim1.br_poziva, y1, "\t", func_sim2.br_poziva, y2 )

pomak: 1 	 44022 [[1. 1.]] 	 1176 [[21. 20.]]
pomak: 2 	 44022 [[1.5 1.5]] 	 1204 [[22. 20.]]
pomak: 3 	 44022 [[2. 2.]] 	 1232 [[23. 20.]]
pomak: 4 	 44022 [[2.5 2.5]] 	 1232 [[24. 20.]]
pomak: 5 	 44022 [[3. 3.]] 	 1260 [[25. 20.]]
pomak: 6 	 44022 [[3.5 3.5]] 	 1260 [[26. 20.]]
pomak: 7 	 44022 [[4. 4.]] 	 1288 [[27. 20.]]
pomak: 8 	 44022 [[4.5 4.5]] 	 1288 [[28. 20.]]
pomak: 9 	 44022 [[5. 5.]] 	 1288 [[29. 20.]]
pomak: 10 	 44022 [[5.5 5.5]] 	 1316 [[30. 20.]]
pomak: 11 	 44022 [[6. 6.]] 	 1316 [[31. 20.]]
pomak: 12 	 44022 [[6.5 6.5]] 	 1316 [[32. 20.]]
pomak: 13 	 44022 [[7. 7.]] 	 1316 [[33. 20.]]
pomak: 14 	 44022 [[7.5 7.5]] 	 1344 [[34. 20.]]
pomak: 15 	 44022 [[8. 8.]] 	 44304 [[35. 20.]]
pomak: 16 	 44022 [[8.5 8.5]] 	 1344 [[36. 20.]]
pomak: 17 	 44022 [[9. 9.]] 	 1344 [[37. 20.]]
pomak: 18 	 44022 [[9.5 9.5]] 	 1344 [[38. 20.]]
pomak: 19 	 44022 [[10. 10.]] 	 44304 [[39. 20.]]
pomak: 20 	 44022 [[10.5 10.5]] 	 1372 [[40. 20.]]


5. Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 4
10−
). 

In [ ]:
from random import randint

iteration = 0
for i in range(50):
    while True:
        iteration += 1
        x1 = randint(-50, 50)
        x2 = randint(-50, 50)
        tocka = Matrica(1, 2, [x1, x2])

        func_sim = Funkcija(f6)
        rez = simplex(tocka, func_sim)

        if f6(rez) < 1e-4: break
(50/iteration) * 50

### Provjere

In [ ]:
# Provjera mnozenja matrica

v1 = Matrica(1, 3, [1, 2, 3])
v2 = Matrica(3, 1, [1, 2, 3])
m1 = Matrica(3, 3, [[1, 2, 3], [1, 1, 1], [1, 2, 3]])

print("m1/3")
print(m1/3)

print("1/3 * (m1)")
print(1/3 * m1)

# print(m1*v1)
print("m1*v2")
print(m1*v2)
print("v1*m1")
print(v1*m1)
print("m1*m1")
print(m1*m1)
print("v1*v2")
print(v1*v2)
print("v2*v1")
print(v2*v1)

In [ ]:
# pomak prema X[l]
sim = [Matrica(1, 2, [2, 3]), Matrica(1, 2, [1, 1]), Matrica(1, 2, [3, 4]), Matrica(1, 2, [4, 5]), Matrica(1, 2, [5, 6])]
novi = pomak_prema_xl(sim, 1)
for m in novi:
    print(m)
    print(m[0][0])

In [ ]:
# provjera nove verzije fje __init__ za matrice 

# three = np.array(3)
# four = np.array(4)
# five = np.array(5)
three = [3]
four = [4]
five = [5]
np.array([three, four, five])
m1 = Matrica(3, 1, [4, 5, 6])
m2 = Matrica(1, 3, [4, 5, 6])
print(m1[0][0])
print(m2[0][0])

In [ ]:
# provjera break-a
gh = 0
while gh < 3:
    gh += 1
    for fj in range(4):
        if gh > 2: break
        print("inside for: gh=" + str(gh))
    print("outside for: gh=" + str(gh))